# Cell 1: Import libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import zscore

plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Cell 2: Load data and initial processing


In [ ]:
df = pd.read_csv('../Data/Resource_utilization.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

# Display basic info
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (26305, 6)


,timestamp,cpu_utilization,memory_usage,storage_usage,workload,Resource Allocation
0,2022-01-01 00:00:00,39.963210,37.120463,77.268825,87.597635,51.911114
1,2022-01-01 01:00:00,86.060728,51.540619,56.655229,100.000000,61.732726
2,2022-01-01 02:00:00,68.566681,NaN,62.418368,100.000000,75.685323
3,2022-01-01 03:00:00,57.903428,51.213742,95.346718,100.000000,67.873120
4,2022-01-01 04:00:00,NaN,74.822334,84.865431,93.433681,65.148501


# Cell 3: Split data into train and test sets


In [ ]:
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size].copy()
test_df = df.iloc[train_size:].copy()

print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")

Training set size: 21044
Testing set size: 5261


# Cell 4: Handle missing values using training statistics only
## Calculate means from training data

In [ ]:
train_means = train_df[['cpu_utilization', 'memory_usage', 'storage_usage',
                        'workload', 'Resource Allocation']].mean()

# Fill missing values in training data
for col in ['cpu_utilization', 'memory_usage', 'storage_usage', 'workload', 'Resource Allocation']:
    train_df[col] = train_df[col].fillna(train_means[col])

# Fill missing values in test data using training statistics
for col in ['cpu_utilization', 'memory_usage', 'storage_usage', 'workload', 'Resource Allocation']:
    test_df[col] = test_df[col].fillna(train_means[col])

print("Missing values after handling:")
print("Training set:")
print(train_df.isnull().sum())
print("\nTesting set:")
print(test_df.isnull().sum())

Missing values after handling:
Training set:
timestamp              0
cpu_utilization        0
memory_usage           0
storage_usage          0
workload               0
Resource Allocation    0
dtype: int64

Testing set:
timestamp              0
cpu_utilization        0
memory_usage           0
storage_usage          0
workload               0
Resource Allocation    0
dtype: int64


# Cell 5: Handle outliers using training statistics only

In [ ]:
for col in ['cpu_utilization', 'memory_usage', 'storage_usage', 'Resource Allocation']:
    # Calculate mean and std from training data only
    train_mean = train_df[col].mean()
    train_std = train_df[col].std()

    # Define threshold (3 standard deviations)
    threshold = 3 * train_std

    # Handle outliers in training data
    train_outliers = np.abs(train_df[col] - train_mean) > threshold
    if train_outliers.sum() > 0:
        train_df.loc[train_outliers, col] = np.nan
        train_df[col] = train_df[col].fillna(train_mean)

    # Handle outliers in test data using training statistics
    test_outliers = np.abs(test_df[col] - train_mean) > threshold
    if test_outliers.sum() > 0:
        test_df.loc[test_outliers, col] = np.nan
        test_df[col] = test_df[col].fillna(train_mean)

print("Outliers handled using training statistics only")

Outliers handled using training statistics only


# Cell 6: Clip values that exceed limits


In [ ]:
metrics_to_clip = {
    'cpu_utilization': 100,
    'storage_usage': 100,
    'workload': 100
}

for metric, upper_limit in metrics_to_clip.items():
    # Clip training data
    train_outlier_count = train_df[train_df[metric] > upper_limit].shape[0]
    if train_outlier_count > 0:
        print(f"Clipping {train_outlier_count} values in training '{metric}' that exceed {upper_limit}.")
        train_df[metric] = np.clip(train_df[metric], 0, upper_limit)

    # Clip test data
    test_outlier_count = test_df[test_df[metric] > upper_limit].shape[0]
    if test_outlier_count > 0:
        print(f"Clipping {test_outlier_count} values in test '{metric}' that exceed {upper_limit}.")
        test_df[metric] = np.clip(test_df[metric], 0, upper_limit)

Clipping 1378 values in training 'storage_usage' that exceed 100.
Clipping 214 values in training 'workload' that exceed 100.
Clipping 49 values in test 'workload' that exceed 100.


# Cell 7: Create basic temporal features

In [ ]:
for dataset in [train_df, test_df]:
    dataset['date_only'] = dataset['timestamp'].dt.date
    dataset['hour_of_day'] = dataset['timestamp'].dt.hour
    dataset['day_of_week'] = dataset['timestamp'].dt.day_name()
    dataset['day_index'] = dataset['timestamp'].dt.dayofweek

print("Basic temporal features created for both datasets")

Basic temporal features created for both datasets


# Cell 8: Create lag features safely

In [ ]:
for dataset in [train_df, test_df]:
    for lag in [1, 2, 3]:
        dataset[f'workload_lag_{lag}'] = dataset['workload'].shift(lag)
        dataset[f'cpu_utilization_lag_{lag}'] = dataset['cpu_utilization'].shift(lag)

# Fill NaN values caused by shifting
train_df = train_df.ffill().bfill()
test_df = test_df.ffill().bfill()

print("Lag features created and NaN values handled")

Lag features created and NaN values handled


# Cell 9: Create rolling features using training statistics only
## Calculate rolling statistics from training data

In [ ]:
workload_rolling_mean = train_df['workload'].shift(1).rolling(window=5).mean().iloc[-1]
workload_rolling_std = train_df['workload'].shift(1).rolling(window=5).std().iloc[-1]
workload_expanding_mean = train_df['workload'].shift(1).expanding().mean().iloc[-1]
cpu_utilization_expanding_mean = train_df['cpu_utilization'].shift(1).expanding().mean().iloc[-1]

# Apply these statistics to both datasets
for dataset in [train_df, test_df]:
    dataset['workload_rolling_mean_3'] = workload_rolling_mean
    dataset['workload_rolling_std_3'] = workload_rolling_std
    dataset['workload_expanding_mean'] = workload_expanding_mean
    dataset['cpu_utilization_expanding_mean'] = cpu_utilization_expanding_mean

print("Rolling features created using training statistics only")

Rolling features created using training statistics only


# Cell 10: Display processed data samples

In [ ]:
print("Training data sample:")
print(train_df.head())
print("\nTesting data sample:")
test_df.head()

Training data sample:
            timestamp  cpu_utilization  memory_usage  storage_usage  \
0 2022-01-01 00:00:00        39.963210     37.120463      77.268825   
1 2022-01-01 01:00:00        86.060728     51.540619      56.655229   
2 2022-01-01 02:00:00        68.566681     51.642745      62.418368   
3 2022-01-01 03:00:00        57.903428     51.213742      95.346718   
4 2022-01-01 04:00:00        52.017738     74.822334      84.865431   

     workload  Resource Allocation   date_only  hour_of_day day_of_week  \
0   87.597635            51.911114  2022-01-01            0    Saturday   
1  100.000000            61.732726  2022-01-01            1    Saturday   
2  100.000000            75.685323  2022-01-01            2    Saturday   
3  100.000000            67.873120  2022-01-01            3    Saturday   
4   93.433681            65.148501  2022-01-01            4    Saturday   

   day_index  workload_lag_1  cpu_utilization_lag_1  workload_lag_2  \
0          5       87.597635 

,timestamp,cpu_utilization,memory_usage,storage_usage,workload,Resource Allocation,date_only,hour_of_day,day_of_week,day_index,workload_lag_1,cpu_utilization_lag_1,workload_lag_2,cpu_utilization_lag_2,workload_lag_3,cpu_utilization_lag_3,workload_rolling_mean_3,workload_rolling_std_3,workload_expanding_mean,cpu_utilization_expanding_mean
21044,2024-05-26 20:00:00,89.073537,82.023029,71.581637,100.000000,59.984967,2024-05-26,20,Sunday,6,100.000000,89.073537,100.000000,89.073537,100.0,89.073537,93.22108,10.289666,90.631239,51.040787
21045,2024-05-26 21:00:00,80.311226,32.940606,83.268913,100.000000,62.032419,2024-05-26,21,Sunday,6,100.000000,89.073537,100.000000,89.073537,100.0,89.073537,93.22108,10.289666,90.631239,51.040787
21046,2024-05-26 22:00:00,50.430174,38.435092,57.087381,86.527146,48.087431,2024-05-26,22,Sunday,6,100.000000,80.311226,100.000000,89.073537,100.0,89.073537,93.22108,10.289666,90.631239,51.040787
21047,2024-05-26 23:00:00,79.776902,60.143070,71.869892,100.000000,68.377242,2024-05-26,23,Sunday,6,86.527146,50.430174,100.000000,80.311226,100.0,89.073537,93.22108,10.289666,90.631239,51.040787
21048,2024-05-27 00:00:00,96.049909,33.641184,89.936842,100.000000,67.763250,2024-05-27,0,Monday,0,100.000000,79.776902,86.527146,50.430174,100.0,80.311226,93.22108,10.289666,90.631239,51.040787


In [24]:
# combine and save processed datasets
processed_data = pd.concat([train_df, test_df], ignore_index=True)
processed_data.to_csv('../Data/Processed_Resource_utilization.csv', index=False)

train_df.to_csv('../Data/Processed_Train_Resource_utilization.csv', index=False)
test_df.to_csv('../Data/Processed_Test_Resource_utilization.csv', index=False)

print("Processed datasets saved successfully!")
print("Training set saved as: ../Data/Processed_Train_Resource_utilization.csv")
print("Testing set saved as: ../Data/Processed_Test_Resource_utilization.csv")

Processed datasets saved successfully!
Training set saved as: ../Data/Processed_Train_Resource_utilization.csv
Testing set saved as: ../Data/Processed_Test_Resource_utilization.csv
